In [ ]:
import pandas as pd
import torch
import numpy as np
from tqdm.auto import tqdm
import random
import os
import re

token = ''
os.environ["OPENAI_API_KEY"] = token

In [ ]:
df = pd.read_csv('../../../../data/2.synthetic_data_generation/generation1/generation1_all_render.csv')
df

In [5]:
system_prompt_template = 'You are an excellent math teacher about to teach students of year group 1 to 14. You will be provided a misconception and a problem with a wrong answer. Your task is to score the quality of the question based on whether the wrong answer is a well designed distractor to correspond with the given misconception.'
user_prompt_template = ''''Observe the following constraints must be adhered to and score the quality of the given question based on the constraints

# Constraints
- Never respond in sentense
- Answer the question with a number from 0 to 5
- The maximum score is 5, and the following criteria will be used to evaluate the scores

# criteria
## The following questions should be scored -5 points.
- The given misconception will not lead to the wrong answer
- The wrong answer given is actually correct

## The following questions should be scored -2 points
- The question asks about a mathmatical concept
- The question is too broad without specific context

## The following questions should be scored -1 points
- There may be multiple wrong answer under the same misconception
- The wrong answer can be caused by other misconception

# Example1
## Input
Misconception: Does not understand the term multiple
Question: Can you give me a few multiples of the number \\( 4 \\)?
Wrong Answer: 8, 12, 16

## The Score is
0

# Example2
## Input
Misconception: Does not know the properties of a linear graph
Question: Can you explain what it means for two lines to be parallel on a graph?
Wrong Answer: Not enough information

## The Score is
3

# Check
## 
Misconception: {misconception}
Question: {question}
Wrong Answer: {wrong_answer}

## The Score is
'''

In [6]:
messages_list = []
for i in tqdm(range(len(df))):
    row = df.iloc[i]
    misconception = row['MisconceptionName']
    question = row['QuestionText']
    wrong_answer = row['AnswerText']

    user_prompt = user_prompt_template.format(misconception=misconception, question=question,wrong_answer=wrong_answer)
    system_prompt = system_prompt_template
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    messages_list.append(messages)

  0%|          | 0/4312 [00:00<?, ?it/s]

In [ ]:
from typing import List
import json

output_file = "misconception_question_generation1_eval.jsonl"

with open(output_file, 'a') as f:
    _id=0
    for messages in messages_list:
        data = {
            "custom_id": f"request-{_id}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4o-mini",
                "messages": messages,
                "max_tokens": 1024
            }
        }
        _id+=1
        f.write(json.dumps(data) + '\n')

In [8]:
messages_list[0:2]

[[{'role': 'system',
   'content': 'You are an excellent math teacher about to teach students of year group 1 to 14. You will be provided a misconception and a problem with a wrong answer. Your task is to score the quality of the question based on whether the wrong answer is a well designed distractor to correspond with the given misconception.'},
  {'role': 'user',
   'content': "'Observe the following constraints must be adhered to and score the quality of the given question based on the constraints\n\n# Constraints\n- Never respond in sentense\n- Answer the question with a number from 0 to 5\n- The maximum score is 5, and the following criteria will be used to evaluate the scores\n\n# criteria\n## The following questions should be scored -5 points.\n- The given misconception will not lead to the wrong answer\n- The wrong answer given is actually correct\n\n## The following questions should be scored -2 points\n- The question asks about a mathmatical concept\n- The question is too br

In [ ]:
from openai import OpenAI
client = OpenAI()

batch_input_file = client.files.create(
  file=open(output_file, "rb"),
  purpose="batch"
)

In [ ]:
batch_input_file_id = batch_input_file.id

batch_job = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "misconception question generation1 eval"
    }
)

Batch(id='batch_67341d35fdd48190a55065172aa055f3', completion_window='24h', created_at=1731468598, endpoint='/v1/chat/completions', input_file_id='file-f7QTtr3t5h2iN6qYup4FUj4M', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1731554998, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'misconception question eval part4'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

# Wait for batch job to complete and retrieve results

In [ ]:
client.batches.retrieve(batch_job.id)

Batch(id='batch_67341d35fdd48190a55065172aa055f3', completion_window='24h', created_at=1731468598, endpoint='/v1/chat/completions', input_file_id='file-f7QTtr3t5h2iN6qYup4FUj4M', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1731472584, error_file_id=None, errors=None, expired_at=None, expires_at=1731554998, failed_at=None, finalizing_at=1731472140, in_progress_at=1731468599, metadata={'description': 'misconception question eval part4'}, output_file_id='file-xFOx5PlXzMn0EMAxjQmTLAal', request_counts=BatchRequestCounts(completed=4312, failed=0, total=4312))

In [ ]:
import json
file_id = 'your-file-id'
file_response = client.files.content(file_id)

lines = file_response1.text.strip().split('\n')

data = [json.loads(line) for line in lines]

In [19]:
output_texts = [d['response']['body']['choices'][0]['message']['content'] for d in data]
output_texts[0:2]

['5', '5']

# Manually clean the scores to -5 to 5 range

In [ ]:
df['quality-gpt4o-mini'] = output_texts_cleaned
df['quality-gpt4o-mini'] = df['quality-gpt4o-mini'].astype(int)

In [ ]:
df.to_csv('../../../../data/2.synthetic_data_generation/generation1/generation1_all_render.csv', index=False)